In [9]:
!jupyter nbconvert --to script SG_Coding_Challenge_v1.ipynb

[NbConvertApp] Converting notebook SG_Coding_Challenge_v1.ipynb to script
[NbConvertApp] Writing 18269 bytes to SG_Coding_Challenge_v1.py


In [7]:
import pandas as pd

# Load the dataset
data = pd.read_csv('bank-additional-full.csv', sep=';')

In [8]:
# Display the first few rows of the dataset
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
# Check the data types and missing values
print(data.info())

# Summary statistics for numerical features
print(data.describe())

# Unique values for categorical features
for col in data.select_dtypes(include=['object']).columns:
    print(f"Unique values for {col}: {data[col].unique()}")

# Distribution of the target variable
print(data['y'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [3]:
# Drop irrelevant columns
data.drop(['duration', 'campaign', 'pdays', 'previous', 'poutcome'], axis=1, inplace=True)

# Convert categorical variables into dummy/indicator variables
data = pd.get_dummies(data)

# Separate features and target variable
X = data.drop('y_yes', axis=1)  # Features
y = data['y_yes']  # Target variable

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)


RandomForestClassifier()

In [5]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7303
           1       1.00      1.00      1.00       935

    accuracy                           1.00      8238
   macro avg       1.00      1.00      1.00      8238
weighted avg       1.00      1.00      1.00      8238

Confusion Matrix:
 [[7303    0]
 [   0  935]]


In [6]:
# Predict probabilities
y_probs = model.predict_proba(X_test)[:, 1]

# Combine probabilities with customer IDs
customer_predictions = pd.DataFrame({'customer_id': X_test.index, 'probability': y_probs})

# Sort customers by probability of subscription
customer_predictions_sorted = customer_predictions.sort_values(by='probability', ascending=False)

# Recommendations: Call customers with high probabilities of subscription
top_customers_to_call = customer_predictions_sorted.head(100)  # Example: Top 100 customers to call
print(top_customers_to_call)

      customer_id  probability
4541        25528         1.00
5918        32645         1.00
1207        30441         1.00
5894        30396         1.00
6067        40482         1.00
...           ...          ...
2176        30017         0.98
3103        40779         0.97
1748        30172         0.97
7712        40008         0.97
1866        40196         0.97

[100 rows x 2 columns]
